# Module 1.1 - Introduction to Intel® oneAPI Deep Neural Network Library (oneDNN)  - Getting Started

## Learning Objectives
In this module the developer will:
* Learn different oneDNN configurations inside the Intel® oneAPI toolkit
* Learn how to compile a oneDNN sample with different configurations via batch jobs on the Intel® DevCloud for oneAPI or in local environments.
* Learn how to program oneDNN with a simple sample


***
# Getting Started Sample Exercise


## introduce oneDNN configurations inside Intel oneAPI toolkits
oneDNN has four different configurations inside the Intel oneAPI toolkits. Each configuration is in a different folder under the oneDNN installation path, and each configurations supports a different compiler or threading library.


Set the installation path of your oneAPI toolkit

In [ ]:
%env ONEAPI_INSTALL=/opt/intel/oneapi

In [ ]:
import os
if os.path.isdir(os.environ['ONEAPI_INSTALL']) == False:
    print("ERROR! wrong oneAPI installation path")

In [ ]:
!printf '%s\n'     $ONEAPI_INSTALL/oneDNN/latest/cpu_*

As you can see, there are four different folders under the oneDNN installation path, and each of those configurations supports different features. This tutorial will show you how to compile and run against different oneDNN configurations.

First, create a lab folder for this exercise.

In [ ]:
!mkdir lab

##  Preparing the getting_started.cpp code

This exercise use the getting_started.cpp example from oneDNN installation path.

First, below section will copy the getting_started.cpp file into lab folder.

In [ ]:
!cp $ONEAPI_INSTALL/oneDNN/latest/cpu_dpcpp_gpu_dpcpp/examples/getting_started.cpp lab/

Users can browser source codes by running below section, and below section also remove comments for readability.

In [ ]:
!cpp -fpreprocessed  -dD -E lab/getting_started.cpp

Then, copy the required header files and CMake file into the lab folder.

In [ ]:
!cp $ONEAPI_INSTALL/oneDNN/latest/cpu_dpcpp_gpu_dpcpp/examples/example_utils.hpp lab/
!cp $ONEAPI_INSTALL/oneDNN/latest/cpu_dpcpp_gpu_dpcpp/examples/example_utils.h lab/
!cp $ONEAPI_INSTALL/oneDNN/latest/cpu_dpcpp_gpu_dpcpp/examples/CMakeLists.txt lab/

# Build and Run with oneAPI DPC++ Compiler 
one of the oneDNN configurations supports oneAPI DPC++ compiler, and it can run on different architectures by using DPC++.
The following section shows you how to build with DPC++ and run on different architectures.

#### Script - build.sh
The script **build.sh** encapsulates the compiler **dpcpp** command and flags that will generate the exectuable.
In order to use DPC++ compiler and related SYCL runtime, some definitions must be passed as cmake arguments.
Here are related cmake arguments for DPC++ configuration : 

   -DCMAKE_C_COMPILER=clang -DCMAKE_CXX_COMPILER=dpcpp -DDNNL_CPU_RUNTIME=SYCL -DDNNL_GPU_RUNTIME=SYCL

In [ ]:
%%writefile build.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --force> /dev/null 2>&1
export EXAMPLE_ROOT=./lab/
mkdir dpcpp
cd dpcpp
cmake .. -DCMAKE_C_COMPILER=clang -DCMAKE_CXX_COMPILER=dpcpp -DDNNL_CPU_RUNTIME=SYCL -DDNNL_GPU_RUNTIME=SYCL
make getting-started-cpp



Once you achieve an all-clear from your compilation, you execute your program on the DevCloud or a local machine.

#### Script - run.sh
the script **run.sh** encapsulates the program for submission to the job queue for execution.
By default, the built program uses CPU as the execution engine, but the user can switch to GPU by giving an input argument "gpu".
The user can refer run.sh below to run on GPU.
To run on CPU, simply remove the input argument "gpu" .

In [ ]:
%%writefile run.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --force > /dev/null 2>&1
echo "########## Executing the run"
./dpcpp/out/getting-started-cpp cpu
echo "########## Done with the run"



#### Submitting **build.sh** and **run.sh** to the job queue
Now we can submit the **build.sh** and **run.sh** to the job queue.
##### NOTE - it is possible to execute any of the build and run commands in local environments.
To enable users to run their scripts both on the Intel DevCloud or in local environments, this and subsequent training checks for the existence of the job submission command **qsub**.  If the check fails, it is assumed that build/run will be local.

In [ ]:
! rm -rf dpcpp;chmod 755 q; chmod 755 build.sh; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q build.sh; ./q run.sh; else ./build.sh; ./run.sh; fi


# Build and Run with GNU Compiler and OpenMP 
One of the oneDNN configurations supports GNU compilers, but it can run only on CPU.
The following section shows you how to build with G++ and run on CPU.

#### Script - build.sh
The script **build.sh** encapsulates the compiler command and flags that will generate the exectuable.
The user must switch to the G++ oneDNN configurations by inputting a custom configuration "--dnnl-configuration=cpu_gomp" when running "source setvars.sh".
In order to use the G++ compiler and related OMP runtime, some definitions must be passed as cmake arguments.
Here are related cmake arguments for DPC++ configuration : 

  -DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++ -DDNNL_CPU_RUNTIME=OMP -DDNNL_GPU_RUNTIME=NONE

In [ ]:
%%writefile build.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_gomp --force> /dev/null 2>&1
export EXAMPLE_ROOT=./lab/
mkdir cpu_gomp
cd cpu_gomp
cmake .. -DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++ -DDNNL_CPU_RUNTIME=OMP -DDNNL_GPU_RUNTIME=NONE
make getting-started-cpp



Once you achieve an all-clear from your compilation, you execute your program on the DevCloud or in local environments.

#### Script - run.sh
the script **run.sh** encapsulates the program for submission to the job queue for execution.
The user must switch to the G++ oneDNN configuration by inputting a custom configuration "--dnnl-configuration=cpu_gomp" when running "source setvars.sh".

In [ ]:
%%writefile run.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_gomp --force> /dev/null 2>&1
echo "########## Executing the run"
./cpu_gomp/out/getting-started-cpp
echo "########## Done with the run"



#### Submitting **build.sh** and **run.sh** to the job queue
Now we can submit the **build.sh** and **run.sh** to the job queue.

##### NOTE - it is possible to execute any of the build and run commands in local environments.
To enable users to run their scripts both on the DevCloud or in local environments, this and subsequent training checks for the existence of the job submission command **qsub**.  If the check fails, it is assumed that build/run will be local.

In [ ]:
! rm -rf cpu_gomp;chmod 755 q; chmod 755 build.sh; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q build.sh; ./q run.sh; else ./build.sh; ./run.sh; fi


# Build and Run with Intel Compiler and OpenMP
One of the oneDNN configurations supports Intel compilers, but it can run only on CPU.
The following section shows you how to build with ICC and run on CPU.

> NOTE : This section is optional and it is for developers who want to use Intel Compiler

#### Script - build.sh
The script **build.sh** encapsulates the compiler command and flags that will generate the executable.
The user must switch to the ICC oneDNN configuration by inputting a custom configuration "--dnnl-configuration=cpu_iomp" when running "source setvars.sh".
In order to use ICC compiler and related OMP runtime, some definitions must be passed as cmake arguments.
Here are related cmake arguments for DPC++ configuration : 

  -DCMAKE_C_COMPILER=icc -DCMAKE_CXX_COMPILER=icpc -DDNNL_CPU_RUNTIME=OMP -DDNNL_GPU_RUNTIME=NONE

In [ ]:
%%writefile build.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_iomp --force> /dev/null 2>&1
export EXAMPLE_ROOT=./lab/
mkdir cpu_iomp
cd cpu_iomp
cmake .. -DCMAKE_C_COMPILER=icc -DCMAKE_CXX_COMPILER=icpc -DDNNL_CPU_RUNTIME=OMP -DDNNL_GPU_RUNTIME=NONE
make getting-started-cpp



Once you achieve an all-clear from your compilation, you execute your program on the DevCloud or in local environments.

#### Script - run.sh
The script **run.sh** encapsulates the program for submission to the job queue for execution.
The user must switch to the ICC oneDNN configuration by inputting a custom configuration "--dnnl-configuration=cpu_iomp" when running "source setvars.sh".

In [ ]:
%%writefile run.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_iomp --force> /dev/null 2>&1
echo "########## Executing the run"
./cpu_iomp/out/getting-started-cpp
echo "########## Done with the run"



#### Submitting **build.sh** and **run.sh** to the job queue
Now we can submit the **build.sh** and **run.sh** to the job queue.

##### NOTE - it is possible to execute any of the build and run commands in local environments.
To enable users to run their scripts both on the DevCloud or in local environments, this and subsequent training checks for the existence of the job submission command **qsub**.  If the check fails it is assumed that build/run will be local.

In [ ]:
! chmod 755 q; chmod 755 build.sh; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q build.sh; ./q run.sh; else ./build.sh; ./run.sh; fi



# Build and Run with GNU Compiler and oneTBB 
One of the oneDNN configurations supports Intel® oneAPI Threading bBuilding block Blocks (oneTBB) as its threading library, but it can run only on CPU.
The following section shows you how to build with oneTBB and run on CPU.

> NOTE : This section is optional and it is for developers who want to use Intel oneTBB

#### Script - build.sh
The script **build.sh** encapsulates the compiler **dpcpp** command and flags that will generate the executable.
The user must switch to the G++ oneDNN configuration by inputting a custom configuration "--dnnl-configuration=cpu_gomp" when running "source setvars.sh".
In order to use G++ compiler and related OMP runtime, some definitions must be passed as cmake arguments.
Here are related cmake arguments for DPC++ configuration : 

  -DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++ -DDNNL_CPU_RUNTIME=TBB -DDNNL_GPU_RUNTIME=NONE

In [ ]:
%%writefile build.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_tbb --force> /dev/null 2>&1
export EXAMPLE_ROOT=./lab/
mkdir cpu_tbb
cd cpu_tbb
cmake .. -DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++ -DDNNL_CPU_RUNTIME=TBB -DDNNL_GPU_RUNTIME=NONE
make getting-started-cpp



Once you achieve an all-clear from your compilation, you execute your program on the DevCloud or a local machine.

#### Script - run.sh
The script **run.sh** encapsulates the program for submission to the job queue for execution.
The user must switch to the oneDNN with oneTBB threading configuration by inputting a custom configuration "--dnnl-configuration=cpu_tbb" when running "source setvars.sh".

In [ ]:
%%writefile run.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_tbb --force> /dev/null 2>&1
echo "########## Executing the run"
./cpu_tbb/out/getting-started-cpp
echo "########## Done with the run"



#### Submitting **build.sh** and **run.sh** to the job queue
Now we can submit the **build.sh** and **run.sh** to the job queue.

##### NOTE - it is possible to execute any of the build and run commands in local environments.
To enable users to run their scripts both on the DevCloud or in local environments, this and subsequent training checks for the existence of the job submission command **qsub**.  If the check fails, it is assumed that build/run will be local.

In [ ]:
! chmod 755 q; chmod 755 build.sh; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q build.sh; ./q run.sh; else ./build.sh; ./run.sh; fi

***
# Summary
In this lab the developer learned the following:
* What are the different oneDNN configurations inside the Intel oneAPI toolkits
* How to compile a oneDNN sample with different configurations via batch jobs on the Intel oneAPI DevCloud or in local environments
* How to program oneDNN with a simple sample
